# Installing and importing packages

In [1]:
! pip install tweepy

    100% |████████████████████████████████| 286kB 7.9MB/s 
  Running setup.py bdist_wheel for PySocks ... - done
  Stored in directory: /content/.cache/pip/wheels/22/5c/b5/12e0dfdfa85bea67b23628b6425fae715c687e947a45ee3df9
Successfully built PySocks


In [2]:
! pip install PyDrive

In [0]:
# Run this cell to import the packages you will need to unpack the dataset
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import pickle
import os
import zipfile
import scipy.ndimage
from google.colab import files

import tweepy
import sys
import json
import codecs
from tweepy.parsers import JSONParser

# Getting new dataset that is cleaned

In [0]:
### authentication -- run once 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
### lists the files in the folder 
file_list = drive.ListFile({'q': "'18sX8Dymv_AAEaY7mJxbmh_LjljD8oRwK' in parents and trashed=false"}).GetList() ## long string in the url of folder
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: Data_2000_2, id: 1sIgmLvS0iJUgxvjKLBjwSQHoY9P-YsyL
title: data_3200.csv, id: 1jOH6hepK10-rdzxv0k6A3_svS4H9T48H
title: Data Vector CLEAN, id: 1uCZsgzLhptTs56JNz_C4lbrbLQvFOAwW
title: Data_2000_1, id: 11fiGbqwp_ekLtPBhuVkjYO3-1RWWr91Y
title: bully_victim1.csv, id: 1ngNQ_sLWIAnN7scAxJl4-y8QZB_soeTQ
title: bully_victim.csv, id: 1qud97IBO3rkLkmDJTi-Y8U--waH7v6tb
title: data_categorical_multi.csv, id: 10qfXUcLUW4GPBM1ra3_cRBpnW9ltQEh-
title: data_categorical_binary.csv, id: 1Mn0cT-De4mq-9gJ7J2s201jvZQ1bWNGK
title: formatted, id: 1cuN_xbf0pz7t1TY6t8vskoVa6SXXWQK7
title: glove.6B.100d.txt, id: 1-FOMH_1xZcjtYo1uW3_2BwozqHFC-dZb
title: Data WITH b', id: 1yDewPt327TxTrKCvuCIpHNyigDMaKPyv
title: glove.6B.zip, id: 1tuFKpS8F1lhnaemAYi_w498Vli6WGuKP
title: data_tweet_clean_500_stop.csv, id: 1bUddQJ0LejpEROhpWmMtYYSoXvR2q4pc
title: Data Cleaning, id: 1JsVnP9kago0ipdSqtX7txLQWMTwSPZ3X
title: data_Tweet_clean.csv, id: 1FNa4-dHR6aEeunRIwMnaBBacSlCHZOsW
title: Medication_1, id: 1yvcgD5F6on7s9gwS56G

In [6]:
### download the file 
file_downloaded = drive.CreateFile({'id': '1FNa4-dHR6aEeunRIwMnaBBacSlCHZOsW'})
file_downloaded.GetContentFile('data_Tweet_clean.csv')

df_data = pd.read_csv('data_Tweet_clean.csv', dtype = object).iloc[:, 1:]
df_data.head(10)

,Tweet ID,User ID,Bullying_Traces?,Type,Form,Teasing?,Author_Role,Emotion
0,107688644067856384,185389094,y,self-disclosure,other,n,defender,empathy
1,102206417217392640,226320672,n,NaN,NaN,NaN,NaN,NaN
2,102779484725448704,297557032,n,NaN,NaN,NaN,NaN,NaN
3,108676972149874688,157724561,n,NaN,NaN,NaN,NaN,NaN
4,106590106873372672,62179998,n,NaN,NaN,NaN,NaN,NaN
5,109034091743154176,177913822,n,NaN,NaN,NaN,NaN,NaN
6,102533497637437441,70412906,y,report,other,n,reporter,none
7,103441158314790912,221102655,y,self-disclosure,other,y,victim,none
8,103679008855691264,11363462,n,NaN,NaN,NaN,NaN,NaN
9,105202476927549441,293593896,n,NaN,NaN,NaN,NaN,NaN


# Authenticating Twitter Process

In [0]:
####### Access Information #################

# Parameter you need to specify
consumer_key = '7w9qPKMQzbCNvx4cjVoaLqjjt'
consumer_secret = 'bj8lNT1yJTkZHlIbg6n7NEBTX73oQttmbPy7nCUuEcYEuldt7O'
access_key = '4439149653-6eK3Z25ODtI3sg7rv3NqNLAWmO6vz270Lewq1tD'
access_secret = 'AJtrhAdfUmWjWnirvASSsp5PEjDbiNwO5XBDxeSuLypxs'

inputFile = 'tweet_id'
outputFile = 'tweet.json'

#############################################

In [0]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth_handler=auth, parser=JSONParser())

# [Not necessary] Just testing out text extraction and adding it as a feature

In [0]:
#extracting the text sample (EXAMPLE)

tweet = api.get_status(int(df_data['Tweet ID'][8]))
#text = tweet['text'].encode('ascii', 'ignore')

In [151]:
type(tweet['user']['location'])
#tweet

str

In [14]:
### concatenating a list of zeros to dataframe
l = pd.Series(list(np.zeros(df_data.shape[0])))
df_data_sample = pd.concat([df_data, l], axis = 1)
df_data_sample = df_data_sample.rename(columns={0: "Text"})
df_data_sample.head(5)

,Tweet ID,User ID,Bullying_Traces?,Type,Form,Teasing?,Author_Role,Emotion,Text
0,107688644067856384,185389094,y,self-disclosure,other,n,defender,empathy,0.0
1,102206417217392640,226320672,n,NaN,NaN,NaN,NaN,NaN,0.0
2,102779484725448704,297557032,n,NaN,NaN,NaN,NaN,NaN,0.0
3,108676972149874688,157724561,n,NaN,NaN,NaN,NaN,NaN,0.0
4,106590106873372672,62179998,n,NaN,NaN,NaN,NaN,NaN,0.0


In [100]:
a = "b\'ABC"
a[2:]

'ABC'

# [Not necessary] Still experimenting

In [129]:
l = pd.Series(list(np.zeros(df_data.shape[0])))
df_data_newcol= pd.concat([df_data, l], axis = 1)
df_data_newcol = df_data_newcol.rename(columns={0: "Text"})
df_data_newcol.shape
df_data_newcol.head(10)

,Tweet ID,User ID,Bullying_Traces?,Type,Form,Teasing?,Author_Role,Emotion,Text
0,107688644067856384,185389094,y,self-disclosure,other,n,defender,empathy,0.0
1,102206417217392640,226320672,n,NaN,NaN,NaN,NaN,NaN,0.0
2,102779484725448704,297557032,n,NaN,NaN,NaN,NaN,NaN,0.0
3,108676972149874688,157724561,n,NaN,NaN,NaN,NaN,NaN,0.0
4,106590106873372672,62179998,n,NaN,NaN,NaN,NaN,NaN,0.0
5,109034091743154176,177913822,n,NaN,NaN,NaN,NaN,NaN,0.0
6,102533497637437441,70412906,y,report,other,n,reporter,none,0.0
7,103441158314790912,221102655,y,self-disclosure,other,y,victim,none,0.0
8,103679008855691264,11363462,n,NaN,NaN,NaN,NaN,NaN,0.0
9,105202476927549441,293593896,n,NaN,NaN,NaN,NaN,NaN,0.0


# ACTUALLY EXTRACTING THE DATA

In [9]:
df_data.shape

(5442, 8)

In [15]:
df_data_part = df_data_sample.iloc[4000:4500]
df_data_part.shape


(500, 9)

In [0]:
text_feature = list()

In [168]:
indices_to_drop = list()
for i in range(3000, 3500):
  try:
    tweet = api.get_status(int(df_data_part['Tweet ID'][i]))
    text = tweet['text'].encode('ascii', 'ignore')
    df_data_part['Text'][i] = text  
  except tweepy.error.TweepError:
    print(tweepy.error.TweepError)
    indices_to_drop.append(i)
    pass

  if i % 50 == 0:
    print(i)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:769: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWit

3000
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
3050
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
3100
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
3150
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepEr

<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>
<class 'tweepy.error.TweepError'>


In [169]:
len(indices_to_drop)

79

## Processed the first 500 

In [170]:
df_data_CLEAN = df_data_part.drop(indices_to_drop)
df_data_CLEAN.head(10)

,Tweet ID,User ID,Bullying_Traces?,Type,Form,Teasing?,Author_Role,Emotion,Text
3000,106325729226137600,79517905,y,report,other,y,reporter,none,b'@Shulinggy u being bullied not new :P'
3001,107184248372477952,229882886,y,report,other,n,reporter,none,"b""@RudeAssYessy i feel you , but if you fight ..."
3002,102334301122732032,44610621,n,NaN,NaN,NaN,NaN,NaN,"b""#News :- 'Financial Services Compensation Sc..."
3003,106400780105433088,360840788,n,NaN,NaN,NaN,NaN,NaN,b'Thompson School District has a new website t...
3004,106870700396122112,210241599,n,NaN,NaN,NaN,NaN,NaN,"b""eu j to sofrendo bullying --' haha @ba_previde"""
3005,102545561076510722,18412950,y,self-disclosure,other,y,bully,none,b'Glad to be back home! I missed my bed (as st...
3006,102807029390901249,232317822,n,NaN,NaN,NaN,NaN,NaN,b'6 - Eu ja sofri Bullying por causa dos menin...
3008,106889862451445760,338761803,n,NaN,NaN,NaN,NaN,NaN,"b""Grieving Mom Forms Group To 'Rise Above' Bul..."
3009,107067427581460480,300217181,n,NaN,NaN,NaN,NaN,NaN,b'No bullying se voc vai forar o meme...'
3010,100961024643039232,351587680,n,NaN,NaN,NaN,NaN,NaN,b'if you are proud to be from birmingham help ...


In [171]:
### removing the b's
for i in range(df_data.shape[0]):
  df_data['Text'][i] = df_data['Text'][i][2:]

KeyError: ignored

In [158]:
type(df_data_CLEAN['Text'][1512])

bytes

In [0]:
### save it to the csv file
df_data_CLEAN.to_csv('data_tweet_clean_500_6.csv')

from google.colab import files
files.download('data_tweet_clean_500_6.csv')

In [12]:
api.wait_on_rate_limit

False